In [ ]:
"Hello world"

In [ ]:
%classpath add mvn cnuernber libpython-clj 1.23

In [ ]:
(require '[libpython-clj.python :as py])

In [ ]:
(py/initialize!)

In [ ]:
(py/run-simple-string "print('printed from python!')")
(println) ;flush *out*. Somehow necessary in the context of the notebook

In [ ]:
;; import vaex
(defonce vaex (py/import-module "vaex"))

In [ ]:
;; df = vaex.example()
(defonce df (py/call-attr vaex "example"))

In [ ]:
df

In [ ]:
;; df_negative = ds[ds.x < 0]
(def df-negative (py/get-item df (py/call-attr (py/get-attr df "x") "__lt__" 0)))
df-negative

In [ ]:
;; First attempt on some syntactic sugar
(require '[clojure.core :as core])

(defonce builtins (py/import-module "builtins"))

(defn expr [df column operator rest]
  (let [lookup {< "__lt__"
                > "__gt__"}
        operator (or (get lookup operator) operator)]
    (py/call-attr
      (py/get-attr df column)
      operator rest)))

(defn df-filter [column operator value df]
  (py/get-item
    df
    (expr df column operator value)))

(defn df-select-columns [col df]
  (py/get-item df col))

(defn slice [df start stop step]
  (py/get-item
    df
    (py/call-attr
      builtins
      "slice" start stop step)))

(defn __getitem__? [obj] (py/has-attr? obj "__getitem__"))

(defn take [n col]
  (if (__getitem__? col)
    (slice col nil, n, nil)
    (core/take n col)))

(defn drop [n col]
  (if (__getitem__? col)
    (slice col n, nil, nil)
    (core/drop n col)))

(defn take-nth [n col]
  (if (__getitem__? col)
    (slice col nil, nil, n) ;; Not yet supported in Vaex
    (core/take-nth n col)))

In [ ]:
;; idiomatic-ish clojure :-)
(->>
    df
    (df-filter "x" < 0)
    (df-filter "y" > 10)
    (drop 1)
    (take 5)
    (df-select-columns ["x" "y"]))